In [1]:
import os
# !!!!!!! Make the API Key external !!!!!!!
os.environ["OPENAI_API_KEY"] = "voc-8162499801266773377505669655d3c05508.40840521"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [3]:
model_name="gpt-3.5-turbo-instruct"
#model_name="gpt-4o-mini"
temperature = 0.0
llm = OpenAI(
    model_name=model_name, temperature=temperature, max_tokens=3500
)

In [ ]:
class PropertySearchRequestClass(BaseModel):
    location: str = Field(
        description = "location in USA including the name the neighborhood"
    )
    style: str = Field(
        description = "style of construction"
    )
    rooms: int = Field(
        description = "number of rooms"
    )
    bedrooms: int = Field(
        description = "number of bedrooms"
    )
    bathrooms: int = Field(
        description = "number of bathrooms"
    )
    floors: int = Field(
        description = "number of floors"
    )
    house_size: int = Field(
        description = "surface area in square feet"
    )
    price: int = Field(
        description = "price in dollars"
    )

In [ ]:
parser = PydanticOutputParser(pydantic_object=PropertySearchRequestClass)

analyse_prompt_generator = PromptTemplate(
    template="{my_query}\n{format_instructions}",
    input_variables=["my_query", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

In [29]:
my_request = "I'm looking for a modern-style house with a sea view for a large family. The house should be located in a quiet neighborhood in San Francisco, California. It should have at least 2 bathrooms and a large garden. The total area of the house should be at least 2500 square feet, and the price should not exceed $500,000."

analyze_query = f"""
    find all available information as defined in the following output schema from this request :"{my_request}", in order to generate a corresponding filled json data structure. set -1 for any missing information.
"""

prompt = analyse_prompt_generator.format(my_query=analyze_query)
analyzed_request = llm.invoke(prompt)
print("Analyzed request: ", analyzed_request)
parsed_response = parser.parse(analyzed_request)
print("Parsed response: ", parsed_response)

Analyzed request:  
{
  "location": "San Francisco, California",
  "style": "modern",
  "rooms": -1,
  "bedrooms": -1,
  "bathrooms": 2,
  "floors": -1,
  "house_size": 2500,
  "price": 500000
}
Parsed response:  location='San Francisco, California' style='modern' rooms=-1 bedrooms=-1 bathrooms=2 floors=-1 house_size=2500 price=500000


In [30]:
data = []
for k, v in vars(parsed_response).items():
    if v != -1:
        data.append(f"{k}: {v}")

data = "; ".join(data)
print(data)

location: San Francisco, California; style: modern; bathrooms: 2; house_size: 2500; price: 500000


In [31]:
reformulate_prompt = f"""generate a simple and clear search request for an home based on the following information: {data}"""
print("Reformulated request: ", reformulate_prompt)

Reformulated request:  generate a simple and clear search request for an home based on the following information: location: San Francisco, California; style: modern; bathrooms: 2; house_size: 2500; price: 500000


In [32]:
reformulated = llm.invoke(reformulate_prompt)
print("reformulated", reformulated)

reformulated 

"I am looking for a modern home in San Francisco, California with 2 bathrooms and a house size of 2500 square feet, priced at $500,000."


In [ ]:
advertisement = """{'location': 'Los Angeles, California', 'style': 'Modern', 'rooms': 6, 'bedrooms': 3, 'bathrooms': 2, 'floors': 2, 'house_size': 2000, 'price': 500000, 'property_description': 'This beautiful modern home is located in the heart of Los Angeles. It features 3 bedrooms, 2 bathrooms, and a spacious living area. The house has 2 floors and a total size of 2000 square feet. It also has a garage and a small garden. The price for this property is $500,000.', 'neighborhood_description': 'The neighborhood is known for its vibrant culture and diverse community. It is also conveniently located near popular restaurants, shopping centers, and entertainment venues.'}"""

response_augmentation_request = f"""reformulate the this advertisement {advertisement} in order to show that it matches the following request: {reformulated}, with respect to the information of the advertisement. Add arguments to the advertisement to make it more attractive. The advertisement should be in a natural language format, and should not be a json object. The advertisement should be at least 100 words long. All the text is in English."""

In [39]:
augmented_advert = llm.invoke(response_augmentation_request)
print("reformulated", augmented_advert)

reformulated 

"Welcome to your dream home in the bustling city of San Francisco, California! This stunning modern property boasts a prime location in the heart of the city, surrounded by the vibrant culture and diverse community that San Francisco is known for. With 3 spacious bedrooms and 2 luxurious bathrooms, this home offers ample space for you and your loved ones to relax and unwind. The 2 floors and 2000 square feet of living space provide the perfect balance of comfort and style. Plus, with a garage and a charming garden, you'll have plenty of room for all your hobbies and outdoor activities. And the best part? This gem is priced at an unbeatable $500,000, making it the perfect opportunity to own your own piece of San Francisco. Don't miss out on this amazing deal - schedule a viewing today!"
